In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import deeplatent as dl

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
     device = torch.device("cpu")
# device = torch.device("cuda:0") 

In [18]:
# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../data/ml-1m/users.dat', sep='::', names=u_cols,
                    encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'genre', 'video_release_date']
movies = pd.read_csv('../data/ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(4),encoding='latin-1')
movies['genre'] = movies.genre.str.split('|')


# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(users, movie_ratings)

In [19]:
lens.head()

,user_id,age,sex,occupation,zip_code,movie_id,title,genre,video_release_date,rating,unix_timestamp
0,1,F,1,10,48067,1,Toy Story (1995),"[Animation, Children's, Comedy]",NaN,5,978824268
1,1,F,1,10,48067,48,Pocahontas (1995),"[Animation, Children's, Musical, Romance]",NaN,5,978824351
2,1,F,1,10,48067,150,Apollo 13 (1995),[Drama],NaN,5,978301777
3,1,F,1,10,48067,260,Star Wars: Episode IV - A New Hope (1977),"[Action, Adventure, Fantasy, Sci-Fi]",NaN,4,978300760
4,1,F,1,10,48067,527,Schindler's List (1993),"[Drama, War]",NaN,5,978824195


In [11]:
y = ratings.rating.values
x = ratings.values[:,0:1].reshape(-1,1)

y = torch.from_numpy(y).to(torch.float64)
x = torch.from_numpy(x).to(torch.int64)

In [12]:
torch.unique(x).shape

torch.Size([75])

In [13]:
model = dl.DeepLatentNN(n_u=3313)

In [14]:

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x).double()
    
    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

2817]],

        ...,

        [[3.3350]],

        [[3.3350]],

        [[3.3350]]], grad_fn=<AddBackward0>)
tensor([[[3.2825]],

        [[3.2825]],

        [[3.2825]],

        ...,

        [[3.3352]],

        [[3.3352]],

        [[3.3352]]], grad_fn=<AddBackward0>)
tensor([[[3.2833]],

        [[3.2833]],

        [[3.2833]],

        ...,

        [[3.3354]],

        [[3.3354]],

        [[3.3354]]], grad_fn=<AddBackward0>)
tensor([[[3.2841]],

        [[3.2841]],

        [[3.2841]],

        ...,

        [[3.3356]],

        [[3.3356]],

        [[3.3356]]], grad_fn=<AddBackward0>)
tensor([[[3.2849]],

        [[3.2849]],

        [[3.2849]],

        ...,

        [[3.3358]],

        [[3.3358]],

        [[3.3358]]], grad_fn=<AddBackward0>)
tensor([[[3.2857]],

        [[3.2857]],

        [[3.2857]],

        ...,

        [[3.3360]],

        [[3.3360]],

        [[3.3360]]], grad_fn=<AddBackward0>)
tensor([[[3.2865]],

        [[3.2865]],

        [[3.2865]],

       